In [1]:
import hashlib
import pandas as pd
import pickle
import os

In [2]:
REPO_DIR = '/home/lyubanenko/data/nghack'

In [3]:
DATA_FILE = os.path.join(REPO_DIR, 'data/intent_train.csv')

OUT_LABELS_FILE = os.path.join(REPO_DIR, 'intent/data/labels.csv')
OUT_TRAIN_FILE = os.path.join(REPO_DIR, 'intent/data/train.bin')
OUT_TEST_FILE = os.path.join(REPO_DIR, 'intent/data/text.bin')

In [4]:
data_ = pd.read_csv(DATA_FILE)

In [5]:
data_[:10]

,id,text,label
0,0,Как отключить тариф?,FAQ - тарифы и услуги
1,1,тариф,мобильная связь - тарифы
2,2,тариф,мобильная связь - тарифы
3,3,Здрасте я хотел получить золотую карту,FAQ - тарифы и услуги
4,4,Золотую карту,FAQ - тарифы и услуги
5,5,Как отключить доступ в интернет,Мобильный интернет
6,6,Как запретить все звонки вне сети оператор,FAQ - интернет
7,7,тарифы для интернета,мобильная связь - тарифы
8,8,тариф,мобильная связь - тарифы
9,9,«тариф»,мобильная связь - тарифы


In [6]:
data = data_.copy()

In [7]:
def clean_text(value):
    value = str(value).strip().lower()
    value = value.strip("“ ”‘ ’«»\"'?!.;: ")
    return value

data['text_clean'] = data['text'].apply(clean_text)

In [8]:
def md5(text):
    h = hashlib.md5()
    h.update(text.encode('utf-8'))
    return h.digest()

data['hash'] = data['text_clean'].apply(md5)

In [15]:
labels_ = sorted(data['label'].unique())

labels = dict()
for i, label in enumerate(labels_):
    fasttext_label = f'__label__{i}'
    labels.update({label: fasttext_label})
    print(f'\'{fasttext_label}\': \'{label}\',')
    
data['fasttext_label'] = data['label'].apply(lambda l: labels.get(l))

with open(OUT_LABELS_FILE, 'w') as out:
    out.write('label,fasttext_label\n')
    for label, fasttext_label in labels.items():
        out.write(f'{label},{fasttext_label}\n')

'__label__0': 'FAQ - интернет',
'__label__1': 'FAQ - тарифы и услуги',
'__label__2': 'SIM-карта и номер',
'__label__3': 'Баланс',
'__label__4': 'Личный кабинет',
'__label__5': 'Мобильные услуги',
'__label__6': 'Мобильный интернет',
'__label__7': 'Оплата',
'__label__8': 'Роуминг',
'__label__9': 'Устройства',
'__label__10': 'запрос обратной связи',
'__label__11': 'мобильная связь - зона обслуживания',
'__label__12': 'мобильная связь - тарифы',
'__label__13': 'тарифы - подбор',


In [10]:
hashs = sorted(data['hash'].unique())
hashs_len = len(hashs)

thresh = 0.8
(train_hashs, test_hashs) = (hashs[:int(thresh*hashs_len)], hashs[int(thresh*hashs_len)+1:])

In [11]:
train = data[data['hash'].isin(train_hashs)]
test = data[data['hash'].isin(test_hashs)]

print(train.shape, test.shape)

(26732, 6) (6067, 6)


In [12]:
pickle.dump(train, open(OUT_TRAIN_FILE, "wb"))
pickle.dump(test, open(OUT_TEST_FILE, "wb"))